# Quantitative Finance with Python

### Alan Moreira, University of Rochester Simon Graduate School of Business

# Notebook 7




### Topics covered
* * *
 * Implementing trading strategies using stock level data
 * Examples: 
    - Size anomaly
    - Momentum
    - Reversals
    - Low volatility
 * Implementation trade-offs
 
   - Liquidity
   
   - Tracking error

In [8]:


import pandas as pd
import numpy as np
import datetime as dt
import wrds
import psycopg2 
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'wrds'

# 1. Download data from WRDS

In [ ]:
## GET DATA

# WE will connect with WRDS a data provider aggregator. You can get a password to it by using your rochester email

# we will get data from CRSP , the leading data provider on historical return data. Data is substantially cleaner 
#than bloomberg which tends to have a lot of mistakes especially pre 2000


###################
# Connect to WRDS #
###################
wrdsdo=True # if false get pre downloaded data from dropbox 
if wrdsdo:
    conn=wrds.Connection()

    ###################
    # CRSP Block      #
    ###################
    # The CRSP data set is where you get return data. Here we are tapping into different databasis. Ons is CRSP.msf, 
    #which has monthly retruns, prices, share outstanding. Information about the different fields can be found at:
    # https://wrds-web.wharton.upenn.edu/wrds/ds/crsp/stock_a/msf.cfm?navId=128 

    # the second database in CRSP is CRSP.msenames which has information about the share code, cusip, exchange code, sector
    # of the firm - basically information that is fixed for a given security

    # this code below merges both these data sets using the permno identifier and the criteria that the dates match
    crsp_mo = conn.raw_sql("""
                      select a.permno, a.permco,a.ret, a.vol, a.shrout, a.prc, a.cfacpr, a.cfacshr, a.date, 
                      b.shrcd, b.exchcd, b.siccd, b.ncusip
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1990' and '12/31/2017'
                      and b.shrcd between 10 and 11
                      """) 
    #crsp_mo.to_csv('E:/Dropbox/Public/Fin418/Data/crsp_monthly.csv')
else:
    # Pick one
    # this is larger data set starting in 1963
  #  url='https://www.dropbox.com/s/5ztd2mdq6dn7cqj/crsp19632017.csv?dl=1'
    # this is a little smalled starting in  1990
    url='https://www.dropbox.com/s/bwjtow60uhdg0vc/crsp19902017.csv?dl=1'
    crsp_m0=pd.read_csv(url,index_col=0)
    
# variables downloaded

# 1. Permno, Permco, ncusip, different types of firm identifiers-- some are firm specific other security specific

# 2. shrco is the type of share: common share, ADR, ETF, ....we will focus on common shares

# 3. exchcd is the code of the exchange where the stock was originally listed

# 4. siccd: Industry code of the firm

# 5. ret, vol, shrout, and prc, are the stock return, trading volume, number of share outstanding, and price

# 6.cfacpr: Cumulative Factor to Adjust Price, allow you to adjust prices for splits 
#   cfacshr: Cumulative Factor to Adjust SHARES, allow you to adjust Shares outstanding for splits 


#7. date is the trading date of the return

#8 .NAMEENDT is a the last effective date of a security's name history structure. 
#It is set to the date preceding the Name Effective Date of the next name structure, the maximum of End of Stock Data, 
#or the Delisting Date of the last name structure.


In [ ]:
crsp_mo.head(10)

### Comments

- Permco is what we will use as the true identifier of a firm, as the others tend to change
- Need to be careful with prices as negative prices have "meaning". Basically means that there was no trading in the closing auction and they are simply an average of the bid and the ask.
- Need to be careful with the units of share outstanding. They are in multiples of 1000
- Some firms have multiple securities, need to aggregate across the securities as strategies typically don't have a view on which particualr security is better to make a bet on

In [ ]:
crsp_mo.info()

## Organize data set and create market value of equity for each firm

In [ ]:
# organize CRSP and create market value of equity for each firm


crsp_m=crsp_mo.copy()# create copy so we do not change the original file
crsp_m[['permco','permno','shrcd','exchcd']]=\
    crsp_m[['permco','permno','shrcd','exchcd']].astype(int)# change variable format to integer
crsp_m['date']=pd.to_datetime(crsp_m['date']) # convert to date


# construction of total firm marke cap (need if you want to value weight)
crsp_m['jdate']=crsp_m['date']+MonthEnd(0) # Line up date to be end of month
crsp_m['p']=crsp_m['prc'].abs()/crsp_m['cfacpr'] # price adjusted to splits, absolute value to deal with "negative" prices
crsp_m['tso']=crsp_m['shrout']*crsp_m['cfacshr']*1e3 # total shares out adjusted- reported in thousands
crsp_m['me'] = crsp_m['p']*crsp_m['tso']/1e6 # market cap in $mil

crsp_summe = crsp_m.groupby(['jdate','permco'])['me'].sum().reset_index()\
    .rename(columns={'me':'me_comp'})
# sum of me across different permno belonging to same permco a given date
crsp_m=pd.merge(crsp_m, crsp_summe, how='inner', on=['jdate','permco'])# merge back with the dataset


In [ ]:
crsp_m.head()

### Size anomaly



In [ ]:
#1. make sure is properly sorted
crsp_m.sort_values(['permco','date'],inplace=True)


In [ ]:
# Keep only what we need

df=crsp_m[['date','permco','me_comp','ret']].copy()
df.head()

In [ ]:
#Lag sorting variable by 1 month

df['me1']=df.groupby('permco')['me_comp'].shift(1)

In [ ]:
# Create portfolios based on size sorts


df['group']=df.groupby(['date'])['me1'].transform(lambda x: pd.qcut(x, 10, labels=False,duplicates='drop'))

df.head()

In [ ]:
# portfolio membership is done. Simply need to use weighting scheme

def wavg(group):
    d = group['ret']
    w = group['me1']
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return np.nan
# We now simply have to  use the above function to construct the portfolio

# the code below applies the function in each date,mom_group group,
# so it applies the function to each of these subgroups of the data set, 
# so it retursn one time-series for each mom_group, as it average the returns of
# all the firms in a given group in a given date
port_vwret = df.groupby(['date','group']).apply(wavg)
port_vwret.head(10)


In [ ]:
port_vwret = port_vwret.reset_index().rename(columns={0:'port_vwret'})# give a name to the new time-seires
port_vwret=port_vwret.set_index(['date','group']) # set indexes
port_vwret=port_vwret.unstack(level=-1)
port_vwret.head(10)

In [ ]:
(port_vwret+1).cumprod().plot(logy=True)

### Momentum anomaly



* here I copied and pasted our code from notebook 6

* how do I change this code to work with stock level data?

In [ ]:
del df['group']

In [ ]:
df.head()

In [ ]:
ngroups=10

DataUR=df.copy()
DataUR=DataUR.set_index(['date'])

DataUR['logret']=np.log(DataUR['ret']+1)
temp=DataUR.groupby('permco')[['logret']].rolling(window=1).sum()
temp =temp.reset_index()
temp['cumret']=np.exp(temp['logret'])-1# transform back in geometric  returns


temp = pd.merge(DataUR.reset_index(), temp[['permco','date','cumret']], how='left', on=['permco','date'])
# merge the 12 month return signal back to the original database
temp['mom']=temp.groupby('permco')['cumret'].shift(1)


mom=temp.sort_values(['date','permco']) # sort by date and firm identifier 
mom=mom.dropna(subset=['mom'], how='any')# drop the row if any of these variables 'mom','ret','me' are missing
mom['mom_group']=mom.groupby(['date'])['mom'].transform(lambda x: pd.qcut(x, ngroups, labels=False,duplicates='drop'))
# create 10 groups each month. Assign membership accroding to the stock ranking in the distribution of trading signal 
#in a given month 
# transform in string the group names
mom=mom.dropna(subset=['mom_group'], how='any')
mom[['mom_group']]='m'+mom[['mom_group']].astype(int).astype(str)


mom=mom[['date','permco','ret','mom_group']]
mom=mom.sort_values(['permco','date']) # resort 

# To form equal weighted portfolios is enough to take means!
port_vwret = mom.groupby(['date','mom_group']).mean()
# row the different dates
port_vwret = port_vwret.reset_index().rename(columns={0:'port_vwret'})# give a name to the new time-seires

port_vwret.head(10)
port_vwret=port_vwret.set_index(['date','mom_group']) # set indexes
port_vwret=port_vwret.ret.unstack(level=-1) 

In [ ]:
(port_vwret+1).cumprod().plot()

In [ ]:

ngroups=10
DataUR=DataR.unstack()
DataUR=pd.DataFrame(DataUR)
DataUR=DataUR.reset_index()
DataUR.rename(columns={'level_0':'ind','level_1':'date',0:'R'},inplace=True)
DataUR=DataUR.set_index(['date'])


DataUR['logret']=np.log(DataUR['R']+1)
temp=DataUR.groupby('ind')[['logret']].rolling(window=12).sum()
temp =temp.reset_index()
temp['cumret']=np.exp(temp['logret'])-1# transform back in geometric  returns


temp = pd.merge(DataUR.reset_index(), temp[['ind','date','cumret']], how='left', on=['ind','date'])
# merge the 12 month return signal back to the original database
temp['mom']=temp.groupby('ind')['cumret'].shift(2)


mom=temp.sort_values(['date','ind']) # sort by date and firm identifier 
mom=mom.dropna(subset=['mom'], how='any')# drop the row if any of these variables 'mom','ret','me' are missing
mom['mom_group']=mom.groupby(['date'])['mom'].transform(lambda x: pd.qcut(x, ngroups, labels=False,duplicates='drop'))
# create 10 groups each month. Assign membership accroding to the stock ranking in the distribution of trading signal 
#in a given month 
# transform in string the group names
mom[['mom_group']]='m'+mom[['mom_group']].astype(int).astype(str)


mom=mom[['date','ind','R','mom_group']]
mom=mom.sort_values(['ind','date']) # resort 

# To form equal weighted portfolios is enough to take means!
port_vwret = mom.groupby(['date','mom_group']).mean()
# row the different dates
port_vwret = port_vwret.reset_index().rename(columns={0:'port_vwret'})# give a name to the new time-seires
port_vwret=port_vwret.set_index(['date','mom_group']) # set indexes
port_vwret=port_vwret.unstack(level=-1) 
port_vwret.head(10)

In [ ]:
# look at cumulative returns plot

In [ ]:
# look at annulaized means

In [ ]:
# look at annualized Sharpe ratios

In [ ]:
# look at winner minus looser: means, sharpe ratios, cumulative returns

### Here is the actual code

In [ ]:


def momreturns(data):
    

    _tmp_crsp = data[['permno','date','ret', 'me','vol','prc']].sort_values(['permno','date']).set_index('date')
    _tmp_crsp['volume']=_tmp_crsp['vol']*_tmp_crsp['prc'].abs()*100/1e6 # in million dollars like me
    ## Trading siginal construction
    _tmp_crsp['ret']=_tmp_crsp['ret'].fillna(0)#replace missing return with 0
    _tmp_crsp['logret']=np.log(1+_tmp_crsp['ret'])#transform in log returns

    _tmp_cumret = _tmp_crsp.groupby(['permno'])['logret'].rolling(12, min_periods=7).sum()# sum last 12 month returns for each 

    
    #stock,require there is a minium of 7 months
    
    _tmp_cumret = _tmp_cumret.reset_index()# reset index, needed to merged back below

    _tmp_cumret['cumret']=np.exp(_tmp_cumret['logret'])-1# transform back in geometric  returns
    _tmp_cumret = pd.merge(_tmp_crsp.reset_index(), _tmp_cumret[['permno','date','cumret']], how='left', on=['permno','date'])
    # merge the 12 month return signal back to the original database

    _tmp_cumret['mom']=_tmp_cumret.groupby('permno')['cumret'].shift(2) # lag the 12 month signal by two months
 
    # You always have to lag by 1 month to make the signal tradable, that is if you are going to use the signal
    #to construct a portfolio in the beggining of january/2018, you can only have returns in the signal up to Dec/2017
    # we are lagging one more time, because the famous momentum strategy skips a month as well 

    # we also labeling mom the trading signal
    mom=_tmp_cumret.sort_values(['date','permno']).drop_duplicates() # sort by date and firm identifier and drop in case there 
    #any duplicates (IT shouldn't be, in a given date for a given firm we can only have one row)

    mom['w']=mom.groupby('permno')['me'].shift(1) # lag the market equity that we will use in our trading strategy to construct
   
    # value-weighted returns
    mom=mom.dropna(subset=['mom','ret','w'], how='any')# drop the row if any of these variables 'mom','ret','me' are missing
    mom['mom_group']=mom.groupby(['date'])['mom'].transform(lambda x: pd.qcut(x, 10, labels=False,duplicates='drop'))

    # create 10 groups each month. Assign membership accroding to the stock ranking in the distribution of trading signal 
    #in a given month 



    # transform in string the group names
    mom[['mom_group']]='m'+mom[['mom_group']].astype(int).astype(str)
    
    mom['date']=mom['date']+MonthEnd(0) #shift all the date to end of the month
    mom=mom.sort_values(['permno','date']) # resort 

    # we now have the membership that will go in each portfolio
    # withing a given portfolio we will simply use the firms market cap to value-weight the portfolio

    # this function takes given date/set of firms given in group, and uses ret_name as the return series and
    # weight_name as the variable to be used for weighting

    # it returns, on number, the value weighted returns

    def wavg(group, ret_name, weight_name):
        d = group[ret_name]
        w = group[weight_name]
        try:
            return (d * w).sum() / w.sum()
        except ZeroDivisionError:
            return np.nan
    # We now simply have to  use the above function to construct the portfolio

    # the code below applies the function in each date,mom_group group,
    # so it applies the function to each of these subgroups of the data set, 
    # so it retursn one time-series for each mom_group, as it average the returns of
    # all the firms in a given group in a given date
    port_vwret = mom.groupby(['date','mom_group']).apply(wavg, 'ret','w')
    port_vwret = port_vwret.reset_index().rename(columns={0:'port_vwret'})# give a name to the new time-seires
    port_vwret=port_vwret.set_index(['date','mom_group']) # set indexes
    port_vwret=port_vwret.unstack(level=-1) # unstack so we have in each column the different portfolios, and in each 
    port_vwret=port_vwret.port_vwret
    
    return port_vwret

In [ ]:
mom=momreturns(crsp_m)
mom.head()

Lets make a flexible code that allow us to modify our trading strategy

1. strategy look back period
2. Skip the first month?
3. Weighting criteria: value weighted, equal weighted, volume weighted
4. return transformation: sum? std? min? max?
5. number of portfolios


In [ ]:
# copy de code here and modify it



# Some insights on Implementation and Liquidity

So we have taken prices as given and discussed quantitative allocation rules that overperform relative to a standard CAPM benchmark.

An important question is wheter we would be able to trade at those prices if we tried to implement the strategy.

Liquidity is the ease of trading a security.

This can be quite complex and encompass many things, all of which impose a cost on those wishing to trade.

Sources of illiquidity:

1. Exogenous transactions costs = brokerage fees, order-processing costs, transaction taxes.

2. Demand pressure = when need to sell quickly, sometimes a buyer is not immediately available (search costs).

3. Inventory risk = if can’t find a buyer will sell to a market maker who will later sell position.  But, since market maker faces future price changes, you must compensate him for this risk.

4. Private information = concern over trading against an informed party (e.g., insider).  Need to be compensated for this. Private information can be about fundamentals or order flow

Obviuosly a perfect answer to this question requires costly experimentation. It would require trading and measuring how prices change in reponse to your trading behavior.

A much simpler approach is to measure the absorption capacity of the strategy.

The idea is to measure how much of the trading volume of each stock you would "use" to implement the strategy for a given position size

Specifically:

$$UsedVolume_{i,t}=\frac{Trading_{i,t}}{Volume_{i,t}}$$

The idea is that if you trade a small share of the volume, you are likely to able to trade at the posted prices.


To compute how much you need to trade you need to compare the desired weights in date $t$ with the weights you have in the end of date $t+1$.

Before trading your weight in date $t+1$ is


$$W_{i,t+1}(before trading)= \frac{W_{i,t}^*(1+R_{i,t+1})}{(1+R_{t+1}^{strategy})}$$


If the desired position in the stock is $$W_{i,t+1}^*$$, then

$$UsedVolume_{i,t}=position\frac{W_{i,t+1}^*-W_{i,t+1}(before trading)}{Volume_{i,t}}$$

which is


$$UsedVolume_{i,t}=\frac{position}{Volume_{i,t}}\left(W_{i,t+1}^*-\frac{W_{i,t}^*(1+R_{i,t+1})}{1+R_{t+1}^{strategy}}\right)$$


$UsedVolume_{i,t}$ is a stock-time specific statistic. Implementability will depend of how high this quantity is across time and across stocks-- the lower the better. 

If it is very high--i.e. close to  1-- means that your position would require almost all volume in a particular stock. It doesn't mean that you wouldn't be able to trade, but likely prices would move against you (i.e., go up as you buy, do down as you sell)

One vere conservative way of looking at it to look at the maximum of this statistic across stocks. This tell you the "weakest" link in your portfolio formation. 

The max statistic for  is the right one to look at if you are unwilling to deviate from your "wish portfolio".

But the "wish portfolio" does not take into account transaction costs

How can portfolios take into account trading costs to reduce total costs substantially?

Can we change the portfolios to reduce trading costs without altering them significantly?

One simple way of looking at this is to look at the  95/75/50 percentiles of the used volume distribution.

If it declines steeply it might makes sense to avoid the to 5%/25% of the stocks that least liquid in you portfolio

But as you deviate from the original portfolio you will have tracking error relative to the original strategy.


In [ ]:
def momreturns_w(data):
    

    _tmp_crsp = data[['permno','date','ret', 'me','vol','prc']].sort_values(['permno','date']).set_index('date')
    _tmp_crsp['volume']=_tmp_crsp['vol']*_tmp_crsp['prc'].abs()*100/1e6 # in million dollars like me

    _tmp_crsp['ret']=_tmp_crsp['ret'].fillna(0)#replace missing return with 0
    _tmp_crsp['logret']=np.log(1+_tmp_crsp['ret'])#transform in log returns

    _tmp_cumret = _tmp_crsp.groupby(['permno'])['logret'].rolling(12, min_periods=12).sum()# sum last 12 month returns for each 
    _tmp_cumret = _tmp_cumret.reset_index()# reset index, needed to merged back below

    _tmp_cumret['cumret']=np.exp(_tmp_cumret['logret'])-1# transform back in geometric  returns
    _tmp_cumret = pd.merge(_tmp_crsp.reset_index(), _tmp_cumret[['permno','date','cumret']], how='left', on=['permno','date'])
    # merge the 12 month return signal back to the original database

    _tmp_cumret['mom']=_tmp_cumret.groupby('permno')['cumret'].shift(2) # lag the 12 month signal by two months
 
 
    mom=_tmp_cumret.sort_values(['date','permno']).drop_duplicates() # sort by date and firm identifier and drop in case there 
    #any duplicates (IT shouldn't be, in a given date for a given firm we can only have one row)

    mom['w']=mom.groupby('permno')['me'].shift(1) # lag the market equity that we will use in our trading strategy to construct
   
    # value-weighted returns
    mom=mom.dropna(subset=['mom','ret','w'], how='any')# drop the row if any of these variables 'mom','ret','me' are missing
    mom['mom_group']=mom.groupby(['date'])['mom'].transform(lambda x: pd.qcut(x, 10, labels=False,duplicates='drop'))
    mom[['mom_group']]='m'+mom[['mom_group']].astype(int).astype(str)
    
    mom['date']=mom['date']+MonthEnd(0) #shift all the date to end of the month
    mom=mom.sort_values(['permno','date']) # resort 


    def wavg(group, ret_name, weight_name):
        d = group[ret_name]
        w = group[weight_name]
        try:
            group['Wght']=(w) / w.sum()
            return group[['date','permno','Wght']]
        except ZeroDivisionError:
            return np.nan
    # We now simply have to  use the above function to construct the portfolio

    # the code below applies the function in each date,mom_group group,
    # so it applies the function to each of these subgroups of the data set, 
    # so it retursn one time-series for each mom_group, as it average the returns of
    # all the firms in a given group in a given date
    
    weights = mom.groupby(['date','mom_group']).apply(wavg, 'ret','me') 
# merge back
    weights=mom.merge(weights,on=['date','permno'])
    weights=weights.sort_values(['date','permno'])


    weights['mom_group_lead']=weights.groupby('permno').mom_group.shift(-1)
    weights['Wght_lead']=weights.groupby('permno').Wght.shift(-1)
    weights=weights.sort_values(['permno','date'])
    
    def wavg(group, ret_name, weight_name):
        d = group[ret_name]
        w = group[weight_name]
        try:
            return (d * w).sum() / w.sum()
        except ZeroDivisionError:
            return np.nan
    
    port_vwret = mom.groupby(['date','mom_group']).apply(wavg, 'ret','w')
    port_vwret = port_vwret.reset_index().rename(columns={0:'port_vwret'})# give a name to the new time-seires
   
    
    weights=weights.merge(port_vwret,how='left',on=['date','mom_group'])
    
    port_vwret=port_vwret.set_index(['date','mom_group']) # set indexes
    port_vwret=port_vwret.unstack(level=-1) # unstack so we have in each column the different portfolios, and in each 
    port_vwret=port_vwret.port_vwret
    return port_vwret, weights

In [ ]:
port_ret, wght=momreturns_w(crsp_m)

In [ ]:
wght.head()

In [ ]:
# construct dollars of each stock that need to be bought (sold) at date t+1
mom=wght.copy()
mom['trade']=mom.Wght_lead*(mom.mom_group==mom.mom_group_lead)-mom.Wght*(1+mom.ret)/(1+mom.port_vwret)


In [ ]:
# per dollar of position how much do you have to trade every month?
mom.groupby(['date','mom_group']).trade.apply(lambda x:x.abs().sum()).loc[:,'m1'].plot()

In [ ]:
# lets look at the most illiquid stocks, the stocks that I will likely have most trouble trading
# lets start by normalizing the amount of trade per stock volume

mom['tradepervol']=mom.trade/mom.volume




In [ ]:
# this allow us to study how much of the volume of each stock I will be "using"
# lets choose a position size, here in millions of dollars , because that is the normalization we used for the volume data
Position=1e3 #(1e3 means one billion dollars)
threshold=0.05
(Position*(mom.groupby(['date','mom_group']).tradepervol.quantile(threshold).loc[:,'m9'])).plot()
(Position*(mom.groupby(['date','mom_group']).tradepervol.quantile(1-threshold).loc[:,'m9'])).plot()

# Tracking error

Let $$W^{wishportfolio}_t$$ and $$W^{Implementationportfolio}_t$$ weights , then consider the following regression

$$W^{Implementationportfolio}_tR_{t+1}=\alpha+\beta W^{wishportfolio}_tR_{t+1}+\epsilon_{t+1}$$

a good implementation portfolio has $\beta=1$ and $\sigma(\epsilon)$ and $\alpha\approx 0$.

So one can think of $|\beta-1|$, $\sigma(\epsilon)$ , and $\alpha$ as three dimensions of tracking error.

The $\beta$ dimension can be more easily correted by levering up and down the tracking portoflio (of possible)

The $\sigma(\epsilon)$ can only be corrected by simply making the implemenetation portoflio more similar to the wish portfolio. The cost of this is not obvious. Really depends how this tracking error relates to other stuff in your portfolio.

$\alpha$ is the important part. The actual cost that you expect to pay to deviate from the wish portfolio

------------------------------

In the industry people typicall refer to tracking error as simply

$$\sigma(W^{Implementationportfolio}_tR_{t+1}- W^{wishportfolio}_tR_{t+1})$$

The volatility of a portfolio that goes long the implementation portfolio and shorts the wish portfolio.

This mixes together $|\beta-1|$, $\sigma(\epsilon)$ and completely ignores $\alpha$


In the end the Implementation portfolio is chosen by trading off  trading costs (market impact) and opportunity cost (tracking error).

So you can simply construct strategies that avoid these 5% less liquid stocks, and see how much your tracking error increases and whether these tracking errors are worth the reduction in trading costs

# How to construct an Implementation portfolio?

- A simple strategy: weight by trading volume -> this make sure that you use the same amount of trading volume across all your positions


- Harder to implement: do not buy stocks that are illiquid now or likely to be illiquid next period. Amounts to add another signal interected to the momentum signal. Only buy if illiquid signal not too strong.

How to change our code to implement the volume-weighted approach?


In [ ]:
def momreturns(data,lookback=12,nmonthsskip=1,wghtvar='me',returntransf='sum',nportfolios=10):
    

    _tmp_crsp = data[['permno','date','ret', 'me','vol','prc']].sort_values(['permno','date']).set_index('date')
    _tmp_crsp['volume']=_tmp_crsp['vol']*_tmp_crsp['prc'].abs()*100/1e6 # in million dollars like me
    ## Trading siginal construction
    _tmp_crsp['ret']=_tmp_crsp['ret'].fillna(0)#replace missing return with 0
    _tmp_crsp['logret']=np.log(1+_tmp_crsp['ret'])#transform in log returns
    if returntransf=='sum':
        _tmp_cumret = _tmp_crsp.groupby(['permno'])['logret'].rolling(lookback, min_periods=np.min([7,lookback])).sum()# sum last 12 month returns for each 
    elif returntransf=='std':
        _tmp_cumret = _tmp_crsp.groupby(['permno'])['logret'].rolling(lookback, min_periods=np.min([7,lookback])).std()# sum last 12 month returns for each 
    elif returntransf=='min':
        _tmp_cumret = _tmp_crsp.groupby(['permno'])['logret'].rolling(lookback, min_periods=np.min([7,lookback])).min()# sum last 12 month returns for each 
    elif returntransf=='max':
        _tmp_cumret = _tmp_crsp.groupby(['permno'])['logret'].rolling(lookback, min_periods=np.min([7,lookback])).max()# sum last 12 month returns for each 
 
    
    
    #stock,require there is a minium of 7 months
    
    _tmp_cumret = _tmp_cumret.reset_index()# reset index, needed to merged back below

    _tmp_cumret['cumret']=np.exp(_tmp_cumret['logret'])-1# transform back in geometric  returns
    _tmp_cumret = pd.merge(_tmp_crsp.reset_index(), _tmp_cumret[['permno','date','cumret']], how='left', on=['permno','date'])
    # merge the 12 month return signal back to the original database

    _tmp_cumret['mom']=_tmp_cumret.groupby('permno')['cumret'].shift(1+nmonthsskip) # lag the 12 month signal by two months
 
    # You always have to lag by 1 month to make the signal tradable, that is if you are going to use the signal
    #to construct a portfolio in the beggining of january/2018, you can only have returns in the signal up to Dec/2017
    # we are lagging one more time, because the famous momentum strategy skips a month as well 

    # we also labeling mom the trading signal
    mom=_tmp_cumret.sort_values(['date','permno']).drop_duplicates() # sort by date and firm identifier and drop in case there 
    #any duplicates (IT shouldn't be, in a given date for a given firm we can only have one row)
    if wghtvar=='ew':
        mom['w']=1
    else:
        mom['w']=mom.groupby('permno')[wghtvar].shift(1) # lag the market equity that we will use in our trading strategy to construct
   
    # value-weighted returns
    mom=mom.dropna(subset=['mom','ret','me'], how='any')# drop the row if any of these variables 'mom','ret','me' are missing
    mom['mom_group']=mom.groupby(['date'])['mom'].transform(lambda x: pd.qcut(x, nportfolios, labels=False,duplicates='drop'))

    # create 10 groups each month. Assign membership accroding to the stock ranking in the distribution of trading signal 
    #in a given month 



    # transform in string the group names
    mom[['mom_group']]='m'+mom[['mom_group']].astype(int).astype(str)
    
    mom['date']=mom['date']+MonthEnd(0) #shift all the date to end of the month
    mom=mom.sort_values(['permno','date']) # resort 

    # we now have the membership that will go in each portfolio
    # withing a given portfolio we will simply use the firms market cap to value-weight the portfolio

    # this function takes given date/set of firms given in group, and uses ret_name as the return series and
    # weight_name as the variable to be used for weighting

    # it returns, on number, the value weighted returns

    def wavg(group, ret_name, weight_name):
        d = group[ret_name]
        w = group[weight_name]
        try:
            return (d * w).sum() / w.sum()
        except ZeroDivisionError:
            return np.nan
    # We now simply have to  use the above function to construct the portfolio

    # the code below applies the function in each date,mom_group group,
    # so it applies the function to each of these subgroups of the data set, 
    # so it retursn one time-series for each mom_group, as it average the returns of
    # all the firms in a given group in a given date
    port_vwret = mom.groupby(['date','mom_group']).apply(wavg, 'ret','w')
    port_vwret = port_vwret.reset_index().rename(columns={0:'port_vwret'})# give a name to the new time-seires
    port_vwret=port_vwret.set_index(['date','mom_group']) # set indexes
    port_vwret=port_vwret.unstack(level=-1) # unstack so we have in each column the different portfolios, and in each 
    port_vwret=port_vwret.port_vwret
    
    return port_vwret

In [ ]:
momportfolios=momreturns(crsp_m,lookback=12,nmonthsskip=1,wghtvar='me',returntransf='sum',nportfolios=10)
momportfoliosvol=momreturns(crsp_m,lookback=12,nmonthsskip=1,wghtvar='vol',returntransf='sum',nportfolios=10)
momportfolios=momportfolios.merge(momportfoliosvol,left_index=True,right_index=True,suffixes=['_me','_vol'])

In [ ]:
momportfolios[['m9_me','m9_vol']].plot()

In [ ]:
# lets look at it's tracking error
y=momportfolios['m9_vol']
x=momportfolios['m9_me']
x=sm.add_constant(x)
results = sm.OLS(y,x).fit()
results.summary()

Observations:

- The beta difference 1.17 vs 1 can be adjusted by taking a smaller position on the implementation portfolio

- The alpha is the actual tracking error loss. How much you expect to loose.

- In this case it is economically quite large -0.0036 vs $\beta E[R]$ of 0.0045

- One calculation that people do is to see how much you are getting for your momentum exposure


In [ ]:
[momportfolios['m9_vol'].mean()/results.params[1]*12,momportfolios['m9_me'].mean()*12]

This is very large. A decay of about 50% in the premium that you earn for trading momentum

But be careful to not over interpret this. We are working today with a very short sample, less than 20 years, for average returns tests that is not much at all.

But beta/residulas are well measured even in fairly short samples.


In addition to that you also have to eat the strategy residual risk

In [ ]:
[results.resid.std(),momportfolios['m9_me'].std()]

It is sizable, about 50% of the original strategy volatility

# How our strategies compare with the momentum factor?



In [ ]:
url = "https://www.dropbox.com/s/9346pp2iu5prv8s/MonthlyFactors.csv?dl=1"
Factors = pd.read_csv(url,index_col=0, 
                         parse_dates=True,na_values=-99)
Factors=Factors/100

Factors=Factors.iloc[:,0:5]
Factors['MKT']=Factors['MKT']-Factors['RF']
Factors=Factors.drop('RF',axis=1)
Factors.mean()

In [ ]:
momportfolios=momportfolios.merge(Factors,left_index=True,right_index=True)
momportfolios.head()

In [ ]:
# things to look at:

#y=momportfolios['m9_me']
#y=momportfolios['m9_vol']
#y=momportfolios['m9_me']-momportfolios['m0_me']


y=momportfolios['m9_me']-momportfolios['m0_me']
x=momportfolios[['MKT','Mom']]
x=sm.add_constant(x)
results = sm.OLS(y,x).fit()
results.summary()

why such a large differcenc even for the strategy that follows the Momentum strategy?

# The devil is in the details: Data Cleaning

Eugene Fama always would tell his students: Garbage in, Garbage out

He was also able to look at a Table of a regression, or summary statistics and immediately tell that the person had 
done something different with the data when cleaning it.


When working with raw data, it is really important to undergo careful work to check if the underlying data is indeed real

CRSP dataset is very clean already, but if you are not careful your analysis will pick up features of the data are not real

- Load on microstructure effects- bid-ask bounces

- Load on tiny/peny stocks

- Load on stocks that are outside of your investment mandate (for example stocks that do not trade in the US)

- Load on other financial instruments (closed-end funds, convertible, prefered shares..)


For example, Daniel and Moskowitz impose the following data requriements


(a). CRSP Sharecode 10 or 11. (i.e, common stocks; no ADRs)

(b). CRSP exchange code of 1, 2 or 3 (NYSE, AMEX or NASDAQ)

(c). Must have price at date t-13

(d). Must have return at date t-2

(e). Must have market cap at date t-1



